In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [ ]:
session = requests.Session()
retry = Retry(connect=5, read=5, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('https://', adapter)

url = 'https://www.basketball-reference.com/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                    'AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/140.0.0.0 Safari/537.36',
    'Accept-Language': 'en'
}

page = session.get(url, headers=headers, timeout=20) 

if page.status_code != 200:
    print(f'Bad status code {page.status_code} for {url}')
    
soup = BeautifulSoup(page.text, 'html.parser')
print('URL:', page.url)

In [ ]:
playoffs_url = urljoin(url, '/playoffs/')

playoffs_page = session.get(playoffs_url, headers=headers, timeout=20)

if playoffs_page.status_code != 200:
    print(f'Bad status code {playoffs_page.status_code} for {playoffs_url}')

playoffs_soup = BeautifulSoup(playoffs_page.text, 'html.parser')
print(f'Playoffs URL:', playoffs_page.url)

In [ ]:
year_links = []

ths = playoffs_soup.find_all('th', {'data-stat': 'year_id'})[:3]

for td in ths:
    if td.a:
        player_link = urljoin(url, td.a['href'])
        year_links.append(player_link)

        print(f'Year Link: {year_links}')

In [ ]:
for year_link in year_links:
    page = requests.get(year_link)
    year_soup = BeautifulSoup(page.text, 'html.parser')
    
    strong_tag = year_soup.find('p').find('strong')
    if strong_tag and strong_tag.text == 'League Champion':
        champion_url = strong_tag.find_next('a')['href']
        print(f'Champion URL:', champion_url)
    else:
        print('League Champion not found!')
        
    
    champion_page = requests.get(champion_url, headers=headers, timeout=20)
    if champion_page.status_code != 200:
        print(f'Bad status code {champion_page.status_code} for {champion_url}')
    champion_soup = BeautifulSoup(champion_page.text, 'html.parser')
    
    div_roster = champion_soup.find_all('div', id='div_roster')
    trs = champion_soup.find_all('tr')[:15]
    
    players_ids = []
    player_names = []
    player_positions = []
    player_heights = []
    player_weights = []
    player_experiences = []

    for td in trs:
        if td.a:
            player_id = td.a['href'].split('/')[2].split('.')[0]
            players_ids.append(player_id)
            
            # Player name
            player_name = td.find('a').text.strip()
            player_names.append(player_name)
            
            # Position
            player_position = td.find('td', {'data-stat': 'pos'}).text.strip()
            player_positions.append(player_position)
            
            # Height
            player_height = td.find('td', {'data-stat': 'height'}).text.strip()
            player_heights.append(player_height)
            
            # Weight
            player_weight = td.find('td', {'data-stat': 'weight'}).text.strip()
            player_weights.append(player_weight)
            
            # Experience
            player_experience = td.find('td', {'data-stat': 'years_experience'}).text.strip()
            player_experiences.append(player_experience)

    print('Player IDs:', players_ids)
    print('Player Names:', player_names)
    print('Player Positions:', player_positions)
    print('Player Heights:', player_heights)
    print('Player Weights:', player_weights)
    print('Player Experiences:', player_experiences)